# activity에서 diff 관련 변수

In [1]:
#### import modules
import pandas as pd
import numpy as np
import re
import seaborn as sns
sns.set(color_codes=True)

In [2]:
#### standardized dataset
train_activity = pd.read_csv('../lite_data/train_activity_lite.csv')
test_activity = pd.read_csv('../lite_data/test_activity_lite.csv')
train_label = pd.read_csv('../lite_data/train_label_lite.csv')

#### restored dataset
raw_train_activity = pd.read_csv('../transformed/real_values_int.csv')
raw_test_activity = pd.read_csv('../transformed/test_real_datas_int.csv')

#### 컬럼 순서 new_id / wk / 나머지....
cols = raw_train_activity.columns
raw_train_activity = raw_train_activity.loc[:,[cols[-1],cols[-2]] + cols[0:-2].tolist()].copy()
raw_test_activity = raw_test_activity.loc[:,[cols[-1],cols[-2]] + cols[0:-2].tolist()].copy()
cols = raw_train_activity.columns

In [3]:
#### 변수 정의 - 처음
## X_train, X_test, y_train, label_map
X_train = train_activity.groupby(by='new_id',sort=False,as_index=False).sum().iloc[:,:1]
X_test = test_activity.groupby(by='new_id',sort=False,as_index=False).sum().iloc[:,:1]

In [4]:
## label dictionary
label_map = {'retained':0,'2month':1,'month':2,'week':3}
y_train = pd.Series([label_map[l] for l in train_label.label])

## 시간 관련 기본 변수 (퀘스트, 채팅, 제작 관련 변수 이외의 변수들...)
stat_cols = ['new_id','wk','duel_cnt','duel_win','partybattle_cnt','partybattle_win','cnt_enter_inzone_solo','cnt_clear_inzone_solo',
             'cnt_enter_inzone_light','cnt_clear_inzone_light','cnt_enter_inzone_skilled','cnt_clear_inzone_skilled','cnt_enter_inzone_normal',
             'cnt_clear_inzone_normal','cnt_enter_raid','cnt_clear_raid','cnt_enter_raid_light','cnt_clear_raid_light','cnt_enter_bam','cnt_clear_bam']

train_time_var = raw_train_activity.loc[:,stat_cols]
test_time_var = raw_test_activity.loc[:,stat_cols]

In [5]:
time_cols = train_time_var.columns.tolist()
num_var = len(time_cols)

In [6]:
my_cols = ['duel','party_battle','inzone_solo','inzone_light','inzone_skilled','inzone_normal','raid','raid_light','bam']

In [7]:
for k,i in enumerate(range(2,num_var,2)):
    train_time = train_time_var.iloc[:,[0,i,i+1]]
    not_play_train = (train_time.iloc[:,1] == 0) & (train_time.iloc[:,2] == 0)
    train_temp = pd.DataFrame(train_time.loc[~ not_play_train,:].groupby(by ='new_id').sum())
    X_train= pd.merge(X_train,pd.DataFrame(train_temp.iloc[:,1] / train_temp.iloc[:,0], columns=[my_cols[k]+'_ratio']), on = 'new_id',how='left')
    X_train[my_cols[k]+'_ratio'][np.isinf(X_train[my_cols[k]+'_ratio'])] = 1.0
    
    test_time = test_time_var.iloc[:,[0,i,i+1]]
    not_play_test = (test_time.iloc[:,1] == 0) & (test_time.iloc[:,2] == 0)
    test_temp = pd.DataFrame(test_time.loc[~ not_play_test,:].groupby(by ='new_id').sum())
    X_test= pd.merge(X_test,pd.DataFrame( test_temp.iloc[:,1] / test_temp.iloc[:,0], columns=[my_cols[k]+'_ratio']), on = 'new_id',how='left')
    X_test[my_cols[k]+'_ratio'][np.isinf(X_test[my_cols[k]+'_ratio'])] = 1.0

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [14]:
X_train.to_csv('X_train_ratio.csv',index = False)
X_test.to_csv('X_test_ratio.csv',index = False)

# activity에서 CAT 말고cnt로?

In [15]:
#### CAT인애들 불러오기
train_CAT = pd.read_csv('X_train_stat.csv')

In [17]:
Cats= [x for x in train_CAT.columns.tolist() if 'CAT' in x]

In [ ]:
stat_cols = ['new_id','wk','duel_cnt','duel_win','partybattle_cnt','partybattle_win','cnt_enter_inzone_solo','cnt_clear_inzone_solo',
             'cnt_enter_inzone_light','cnt_clear_inzone_light','cnt_enter_inzone_skilled','cnt_clear_inzone_skilled','cnt_enter_inzone_normal',
             'cnt_clear_inzone_normal','cnt_enter_raid','cnt_clear_raid','cnt_enter_raid_light','cnt_clear_raid_light','cnt_enter_bam','cnt_clear_bam']

In [18]:
Cats

['payment_amount_CAT',
 'cnt_dt_CAT',
 'play_time_CAT',
 'game_combat_time_CAT',
 'get_money_CAT',
 'cnt_use_buffitem_CAT',
 'npc_exp_CAT',
 'npc_hongmun_CAT',
 'quest_exp_CAT',
 'quest_hongmun_CAT',
 'item_hongmun_CAT',
 'duel_cnt_CAT',
 'partybattle_cnt_CAT',
 'cnt_enter_inzone_solo_CAT',
 'cnt_enter_inzone_light_CAT',
 'cnt_enter_inzone_skilled_CAT',
 'cnt_enter_inzone_normal_CAT',
 'cnt_enter_raid_CAT',
 'cnt_enter_raid_light_CAT',
 'normal_chat_CAT',
 'whisper_chat_CAT',
 'district_chat_CAT',
 'party_chat_CAT',
 'guild_chat_CAT',
 'making_cnt_CAT',
 'cnt_enter_bam_CAT',
 'faction_chat_CAT',
 'gathering_cnt_CAT']

In [2]:
#### standardized dataset
train_activity = pd.read_csv('../lite_data/train_activity_lite.csv')
test_activity = pd.read_csv('../lite_data/test_activity_lite.csv')
train_label = pd.read_csv('../lite_data/train_label_lite.csv')

#### restored dataset
raw_train_activity = pd.read_csv('../transformed/real_values_int.csv')
raw_test_activity = pd.read_csv('../transformed/test_real_datas_int.csv')

#### 컬럼 순서 new_id / wk / 나머지....
cols = raw_train_activity.columns
raw_train_activity = raw_train_activity.loc[:,[cols[-1],cols[-2]] + cols[0:-2].tolist()].copy()
raw_test_activity = raw_test_activity.loc[:,[cols[-1],cols[-2]] + cols[0:-2].tolist()].copy()
cols = raw_train_activity.columns